In [1]:
#Librerie utili
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import scipy.stats as stats


In [8]:
excel2krSheet=pd.read_excel("../excel/2kr.xlsx")

print(excel2krSheet)

      Unnamed: 0            I             A           B          C         AB  \
0            NaN            1             1    1.000000          1   1.000000   
1            NaN            1             1   -1.000000         -1  -1.000000   
2            NaN            1             1    1.000000         -1   1.000000   
3            NaN            1             1   -1.000000          1  -1.000000   
4            NaN            1            -1    1.000000          1  -1.000000   
5            NaN            1            -1   -1.000000         -1   1.000000   
6            NaN            1            -1    1.000000         -1  -1.000000   
7            NaN            1            -1   -1.000000          1   1.000000   
8            sum      19.6691       6.68689   11.492222    11.4443   6.676000   
9             qi      2.45864      0.835861    1.436528    1.43054   0.834500   
10  8*qi^2 (Ssx)          NaN       55.8931  165.088965    163.716  55.711220   
11     variation          Na

In [4]:
  N = 1200
  p = 0.53
  q = 1000
  obs = np.random.binomial(N, p, size = q)/N

import scipy.stats as stats

z = (obs-np.mean(obs))/np.std(obs)

stats.probplot(z, dist="t", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

TypeError: _parse_args() missing 1 required positional argument: 'df'